# Kelp Project - Data Cleaning

### Tanner Thompson - Math 404 - 3/24/16

In [1]:
%matplotlib inline
import spectral as spy
import spectral.io.envi as envi
import numpy as np
import os
import subprocess
from matplotlib import pyplot as plt
import time

## Data Cleaning!

First, we need to define a method to convert from image coordinates to earth coordinates.

In [4]:
def get_coords(img,x,y):
    """
    Accepts:
        img: a SpyFile object
        x, y: the pixel coordinates (zero-based indexing)
        
    Returns:
        e, n: the UTM coordinates
        z: the UTM zone
        
    Note: the code now accounts for rotation!
    """
    info = img.metadata['map info']
    
    # check the metadata to make sure the format is supported 
    if info[0] != 'UTM':
        raise ValueError('Coordinate generation is only built for UTM, not ' + info[0])
    if info[1] != '1':
        raise ValueError('this should be 1')
    if info[2] != '1':
        raise ValueError('this should be 1')
    if not info[-1].startswith('rotation'):
        raise ValueError('No rotation angle found')
    
    # name some variables to make the code more readable
    easting = float(info[3])
    northing = float(info[4])
    x_pixel_size = float(info[5])
    y_pixel_size = float(info[6])
    utm_zone = int(info[7])
    exec(images[i].metadata['map info'][-1]) # rotation = <rotation angle>
    rotation *= np.pi/180 # convert rotation angle to radians
    
    # do some trigonometry
    # account for the rotation of the image
    new_easting = easting + x*x_pixel_size*np.cos(rotation) + y*y_pixel_size*np.sin(rotation)
    new_northing = northing - y*y_pixel_size*np.cos(rotation) + x*x_pixel_size*np.sin(rotation)
    return new_easting, new_northing, utm_zone

The AVIRIS data comes in a .tar.gz file, which needs to be unzipped.  This code unzips any images that haven't already been unzipped.

In [ ]:
if 'unpacked_tars' not in os.listdir('.'):
    os.makedirs('unpacked_tars')

unpacked_flight_names = map(lambda s: s[:s.index('_')],os.listdir('./unpacked_tars/'))
packed_flight_names = map(lambda s: s[:s.index('_')],os.listdir('./raw_AVIRIS_files/'))

# ONLY WORKS ON LINUX
for flight_name in packed_flight_names:
    if flight_name not in unpacked_flight_names:
        print 'unpacking', flight_name
        subprocess.call('tar -xvzf ./raw_AVIRIS_files/' + flight_name + '_refl.tar.gz -C ./unpacked_tars/', shell = True)
        unpacked_flight_names.append(flight_name)

Finally, we're ready to do some data processing.  To open the AVIRIS image files, we'll use a package called Spectral Python, which was built for opening and visualizing hyperspectral images.

This code opens each image file as a SpyFile object, instead of just a numpy array, because of the size of the file.  This class allows you to read one band or one pixel at a time to avoid loading the entire image (sometimes over 10GB) into the memory at once.

After opening the images, we read in certain important bands. These will be used in identifying the kelp and later in visualization.

The brightness values in the data are dimensionless, as they are the ratio of reflected radiation intensity to received radiation intensity.  Since this gives only values between 0 and 1, the final values have been scaled by a factor of 10000.

We identify the kelp through several different criteria.  First, we calculate the NDVI (Normalized Difference Vegetation Index). High NDVI values can indicate vegetation, but can also indicate a pixel with a zero in the red band (560 nm). To correct for that, we rule out any pixels with a reflectance value less than 40 in the SWIR 1 band (Shortwave Infrared, 1102 nm).  This eliminates virtually all ocean pixels, since water reflects almost no SWIR radiation. Next, we rule out any pixels with a reflectance value greater than 600 in the SWIR 1 band, which eliminates all but the darkest vegetation on land.  Finally, we throw out any remaining pixels with a green reflectance value of less than 60, which gets rid of the rest of the land vegetation.  Thus, we are left with ocean vegetation, which is exclusively kelp.

Once the pixels have been identified, their reflectance vectors are stored in master_pixel_list.

In [7]:
folder_names = sorted(os.listdir('./unpacked_tars/'))

# skip any necessary files - for example, if they're too big 

flights_to_skip = ['f130410t01p00r11']
for flight_to_skip in flights_to_skip:
    for folder_name in folder_names:
        if folder_name.startswith(flight_to_skip):
            folder_names.remove(folder_name)
    
    
master_pixel_list = []
master_coord_list = []
images = []
time1 = time.time()
for i, folder_name in enumerate(folder_names):

    # reads in the image
    print 'reading in', folder_name
    for suffix in ['_corr_v1', 'rdn_refl_img_corr', '_corr_v1d_img']:
        try:
            flight_name = folder_name[:folder_name.index('_')]
            filename = './unpacked_tars/' + folder_name + '/' + flight_name + suffix
            images.append(envi.open(filename + '.hdr'))
        except spy.io.spyfile.FileNotFoundError as e:
            #print e
            pass
        
    # some of the images are scaled differently (the 2016 ones)
    multiplier = 10000 if folder_name.endswith('v1d') else 1
        
    # grabs all the necessary bands and calculates NDVI (normalized difference vegetation index)
    print 'grabbing bands'
    kelpgreen = images[i].read_band(23).astype(float) * multiplier
    red = images[i].read_band(33).astype(float) * multiplier
    nir = images[i].read_band(39).astype(float) * multiplier
    nir2 = images[i].read_band(59).astype(float) * multiplier
    swir1 = images[i].read_band(78).astype(float) * multiplier
    swir2 = images[i].read_band(137).astype(float) * multiplier
    NDVI = ((nir - red)/(nir + red))
    NDVI[np.isnan(NDVI)] = -1
    ratio = nir2/nir

    # here's the magic
    # uses the following criteria to pick the kelp out

    # NDVI == 1 grabs only the most vegetated areas (but also inadverently grabs places where red just is zero)
    # swir1 > 40 gets rid of random pixels in the ocean that have red == 0
    # swir1 < 600 gets rid of some vegatation on land
    # kelpgreen > 60 gets rid of some vegetated pixels on land
    kelp = (NDVI >.6) & (40 < swir1) & (swir1 < 600) & (kelpgreen > 60) & (ratio < .8)

    # converts the boolean array to an array of kelp locations (in image coordinates)
    print 'grabbing pixels'
    kelp_image_coords = np.argwhere(kelp)
    
    # iterates through all True values in kelp and pulls the brightness vectors at each pixel
    # makes an ndarray out of them
    pixels = np.array([images[i].read_pixel(*coordpair) for coordpair in kelp_image_coords])
    
    print 'grabbing coordinates'
    # converts image coordinates to UTM coordinates
    kelp_UTM_coords = np.array(map(lambda c: get_coords(images[i],*c), kelp_image_coords))
    
    
    #adds the newly acquired pixel list and coord list to the master pixel list and coord list
    if len(pixels) > 0:
        master_pixel_list.append(pixels)
        master_coord_list.append(kelp_UTM_coords)
    else:
        print 'PROBLEM:', folder_name, 'has no kelp'
        
print 'done'
time2 = time.time()
print 'took %f seconds' % (time2 - time1)

reading in f130410t01p00r09_refl
grabbing bands
grabbing pixels
grabbing coordinates


/home/tannert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:40: RuntimeWarning: invalid value encountered in divide
/home/tannert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:42: RuntimeWarning: divide by zero encountered in divide
/home/tannert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:42: RuntimeWarning: invalid value encountered in divide
/home/tannert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:51: RuntimeWarning: invalid value encountered in less


reading in f130410t01p00r10_refl
grabbing bands
grabbing pixels
grabbing coordinates
reading in f130411t01p00r12_refl
grabbing bands
grabbing pixels
grabbing coordinates
reading in f130606t01p00r15_refl
grabbing bands
grabbing pixels
grabbing coordinates
reading in f130607t01p00r15_refl
grabbing bands
grabbing pixels
grabbing coordinates
reading in f141124t01p00r17_refl
grabbing bands


/home/tannert/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:40: RuntimeWarning: divide by zero encountered in divide


grabbing pixels
grabbing coordinates
reading in f160617t01p00r15_refl_v1d
grabbing bands
grabbing pixels
grabbing coordinates
reading in f160620t01p00r15_refl_v1d
grabbing bands
grabbing pixels
grabbing coordinates
done
took 121.784326 seconds


In [8]:
pixels = np.vstack(master_pixel_list)
coords = np.vstack(master_coord_list)

In [9]:
pixels.shape

(24018, 224)

In [10]:
np.savetxt('pixels.txt',pixels)
np.savetxt('coords.txt',coords)